In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras

from keras.src.applications import imagenet_utils
# For versions <TF2.13 change the above import to:
# from keras.applications import imagenet_utils

from tensorflow import keras

import tensorflow_datasets as tfds
import tensorflow_addons as tfa

tfds.disable_progress_bar()
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers.legacy import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Hyperparameters

In [ ]:
# Values are from table 4.
patch_size = 4  # 2x2, for the Transformer blocks.
image_size = 256
expansion_factor = 2  # expansion factor for the MobileNetV2 blocks.

In [ ]:
def conv_block(x, filters=16, kernel_size=3, strides=2):
    conv_layer = layers.Conv2D(filters, kernel_size, strides=strides, activation=tf.nn.swish, padding="same")
    return conv_layer(x)


# Reference: https://git.io/JKgtC


def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    m = layers.Conv2D(expanded_channels, 3, padding="same", use_bias=False)(x)        #output channel changed
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    if strides == 2:
        m = layers.ZeroPadding2D(padding=imagenet_utils.correct_pad(m, 3))(m)
    m = layers.DepthwiseConv2D(
        3, strides=strides, padding="same" if strides == 1 else "valid", use_bias=False
    )(m)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    m = layers.Conv2D(output_channels, 4, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)

    if tf.math.equal(x.shape[-1], output_channels) and strides == 1:
        return layers.Add()([m, x])
    return m


# Reference:
# https://keras.io/examples/vision/image_classification_with_vision_transformer/


def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.swish)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=[x.shape[-1] * 2, x.shape[-1]], dropout_rate=0.1,)
        # Skip connection 2.
        x = layers.Add()([x3, x2])

    return x


def mobilevit_block(x, num_blocks, projection_dim, strides=1):
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(
        local_features, filters=projection_dim, kernel_size=1, strides=strides
    )

    # Unfold into patches and then pass through Transformers.
    num_patches = int((local_features.shape[1] * local_features.shape[2]) / patch_size)
    non_overlapping_patches = layers.Reshape((patch_size, num_patches, projection_dim))(local_features)
    global_features = transformer_block(non_overlapping_patches, num_blocks, projection_dim)


    # Fold into conv-like feature-maps.
    folded_feature_map = layers.Reshape((*local_features.shape[1:-1], projection_dim))(global_features)

    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides)
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])
    # Fuse the local and global features using a convoluion layer.
    local_global_features = conv_block(local_global_features, filters=projection_dim, strides=strides)

    return local_global_features

In [ ]:
def create_mobilevit(num_classes=3):
    inputs = keras.Input((image_size, image_size, 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Initial conv-stem -> MV2 block.
    x = conv_block(x, filters=16)
    x = inverted_residual_block(x, expanded_channels=16 * expansion_factor, output_channels=16)

    # Downsampling with MV2 block.
    x = inverted_residual_block(x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2)
    x = inverted_residual_block(x, expanded_channels=24 * expansion_factor, output_channels=24)
    x = inverted_residual_block(x, expanded_channels=24 * expansion_factor, output_channels=24)

    # First MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=48, strides=2
    )
    x = mobilevit_block(x, num_blocks=2, projection_dim=64)

    # Second MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=64 * expansion_factor, output_channels=64, strides=2
    )
    x = mobilevit_block(x, num_blocks=4, projection_dim=80)

    # Third MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=80 * expansion_factor, output_channels=80, strides=2
    )
    x = mobilevit_block(x, num_blocks=3, projection_dim=96)
    x = conv_block(x, filters=320, kernel_size=1, strides=1)

    # Classification head.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    #return keras.Model(inputs, outputs)
    model = keras.Model(inputs, outputs)
    return model

mobilevit_xxs = create_mobilevit()
mobilevit_xxs.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 256, 256, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 16)         448       ['rescaling[0][0]']           
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 32)         4608      ['conv2d[0][0]']              
                                                                                              

In [ ]:
batch_size = 64
auto = tf.data.AUTOTUNE
resize_bigger = 280
num_classes = 3


def preprocess_dataset(is_training=True):
    def _pp(image, label):
        if is_training:
            # Resize to a bigger spatial resolution and take the random
            # crops.
            image = tf.image.resize(image, (resize_bigger, resize_bigger))
            image = tf.image.random_crop(image, (image_size, image_size, 3))
            image = tf.image.random_flip_left_right(image)
        else:
            image = tf.image.resize(image, (image_size, image_size))
        label = tf.one_hot(label, depth=num_classes)
        return image, label

    return _pp


def prepare_dataset(dataset, is_training=True):
    if is_training:
        dataset = dataset.shuffle(batch_size * 10)
    dataset = dataset.map(preprocess_dataset(is_training), num_parallel_calls=auto)
    return dataset.batch(batch_size).prefetch(auto)


## Load and prepare the dataset

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive

# Define dataset path
dataset_path = "/content/drive/Shareddrives/4IR_Research_Students/Sumaya/Potato_ViT/data"

# Load the dataset using tf.keras.preprocessing.image_dataset_from_directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels="inferred",
    label_mode="categorical",  # Update label_mode to "categorical"
    batch_size=64,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training",
) ##################################### batch_size changed

# Split the dataset into training and validation subsets
train_dataset = dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels="inferred",
    label_mode="categorical",  # Update label_mode to "categorical"
    batch_size=64,
    image_size=(256, 256),
    shuffle=False,
    seed=123,
    validation_split=0.1,
    subset="validation",
)
# Assuming prepare_dataset is defined somewhere
def prepare_dataset(dataset, is_training):
    # Your preprocessing code here
    return dataset

# Use prepare_dataset function
train_dataset = prepare_dataset(train_dataset, is_training=True)
val_dataset = prepare_dataset(val_dataset, is_training=False)

Found 3000 files belonging to 3 classes.
Using 2700 files for training.
Found 3000 files belonging to 3 classes.
Using 300 files for validation.


In [ ]:
from tensorflow.keras.optimizers.legacy import Adam

## Train a MobileViT (XXS) model

In [ ]:
learning_rate = 0.002
learning_rate = 0.001
weight_decay = 0.0001
label_smoothing_factor = 0.1
#epochs = 1

#optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing_factor)


def run_experiment(mobilevit_xxs):
    #mobilevit_xxs = create_mobilevit(num_classes=num_classes)
    mobilevit_xxs.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = (mobilevit_xxs.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks=[checkpoint_callback],))
    mobilevit_xxs.load_weights(checkpoint_filepath)
    _, accuracy = mobilevit_xxs.evaluate(val_dataset)
    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    return mobilevit_xxs


#mobilevit_xxs = run_experiment()

In [ ]:
vit_classifier = create_mobilevit()
vit_history = run_experiment(vit_classifier)

Epoch 1/100
43/43 [==============================] - 322s 5s/step - loss: 0.5535 - accuracy: 0.8633 - val_loss: 0.7050 - val_accuracy: 1.0000
Epoch 2/100
43/43 [==============================] - 39s 870ms/step - loss: 0.4356 - accuracy: 0.9259 - val_loss: 0.5559 - val_accuracy: 1.0000
Epoch 3/100
43/43 [==============================] - 38s 858ms/step - loss: 0.3731 - accuracy: 0.9615 - val_loss: 1.1233 - val_accuracy: 0.0000e+00
Epoch 4/100
43/43 [==============================] - 39s 866ms/step - loss: 0.3742 - accuracy: 0.9626 - val_loss: 0.3088 - val_accuracy: 1.0000
Epoch 5/100
43/43 [==============================] - 38s 856ms/step - loss: 0.3507 - accuracy: 0.9737 - val_loss: 0.3065 - val_accuracy: 1.0000
Epoch 6/100
43/43 [==============================] - 38s 856ms/step - loss: 0.3718 - accuracy: 0.9615 - val_loss: 0.2932 - val_accuracy: 1.0000
Epoch 7/100
43/43 [==============================] - 40s 879ms/step - loss: 0.3322 - accuracy: 0.9815 - val_loss: 0.2932 - val_accurac

In [ ]:
# Serialize the model as a SavedModel.
mobilevit_xxs.save("mobilevit_xxs")

# Convert to TFLite. This form of quantization is called
# post-training dynamic-range quantization in TFLite.
converter = tf.lite.TFLiteConverter.from_saved_model("mobilevit_xxs")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS,  # Enable TensorFlow ops.
]
tflite_model = converter.convert()
open("mobilevit_xxs.tflite", "wb").write(tflite_model)

2139800

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Define your list of class names.
class_names = ["Healthy", "Potato_Early_blight", "Potato_Late_blight"]  # Add your class names here

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/mobilevit_xxs.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the input image.
input_shape = input_details[0]['shape']
image = Image.open("/content/healthy.JPG")  # Replace "input_image.jpg" with your image path
image = image.resize((input_shape[1], input_shape[2]))
image = np.asarray(image) / 255.0  # Normalize pixel values
image = image.astype(np.float32)  # Convert to FLOAT32
image = np.expand_dims(image, axis=0)  # Add batch dimension

# Run inference.
interpreter.set_tensor(input_details[0]['index'], image)
interpreter.invoke()

# Get the output tensor.
output = interpreter.get_tensor(output_details[0]['index'])

# Post-process the output to get the predicted class name.
predicted_class_index = np.argmax(output)
predicted_class_name = class_names[predicted_class_index]

# Print the prediction.
print("Predicted Class:", predicted_class_name)

Predicted Class: Potato_Late_blight
